In [ ]:
# Planning an Event for Top-Ten Customers - Assess-Visitors.py 

In [25]:
import sys
import os
import sqlite3 as sq
import pandas as pd
from pandas.io import sql

In [4]:
InputFileName='Retrieve_Online_Visitor.csv'
InputFileDir = 'C:/VKHCG/02-Krennwallner/01-Retrieve/01-EDS/02-Python'
DataBaseDir = 'C:/VKHCG/02-Krennwallner/02-Assess/SQLite'
DataBaseName='krennwallner.db'

In [6]:
if not os.path.exists(DataBaseDir):    
    os.makedirs(DataBaseDir) 

sDatabaseName=DataBaseDir + '/' + DataBaseName
conn = sq.connect(sDatabaseName)

In [8]:
### Import Country Data
sFileName= InputFileDir + '/' + InputFileName
print('Loading :',sFileName)

Loading : C:/VKHCG/02-Krennwallner/01-Retrieve/01-EDS/02-Python/Retrieve_Online_Visitor.csv


In [9]:
VisitorRawData=pd.read_csv(sFileName,
                           header=0,
                           low_memory=False, 
                           encoding="latin-1",
                           skip_blank_lines=True)
VisitorRawData.columns.values

array(['Country', 'Place_Name', 'Latitude', 'Longitude',
       'First_IP_Number', 'Last_IP_Number'], dtype=object)

In [10]:
VisitorRawData.shape

(1247464, 6)

In [11]:
VisitorRawData.drop_duplicates(subset=None, keep='first', inplace=True)
VisitorData=VisitorRawData
print('Loaded Company :',VisitorData.columns.values)

Loaded Company : ['Country' 'Place_Name' 'Latitude' 'Longitude' 'First_IP_Number'
 'Last_IP_Number']


In [12]:
VisitorData.head()

,Country,Place_Name,Latitude,Longitude,First_IP_Number,Last_IP_Number
0,BW,Gaborone,-24.6464,25.9119,692781056.0,692781567.0
1,BW,Gaborone,-24.6464,25.9119,692781824.0,692783103.0
2,BW,Gaborone,-24.6464,25.9119,692909056.0,692909311.0
3,BW,Gaborone,-24.6464,25.9119,692909568.0,692910079.0
4,BW,Gaborone,-24.6464,25.9119,693051392.0,693052415.0


In [13]:
VisitorData.shape

(1247464, 6)

In [14]:
sTable='Assess_Visitor'
print('Storing :',sDatabaseName,' Table:',sTable)
VisitorData.to_sql(sTable, conn, if_exists="replace")

Storing : C:/VKHCG/02-Krennwallner/02-Assess/SQLite/krennwallner.db  Table: Assess_Visitor


In [15]:
print(VisitorData.head())

  Country Place_Name  Latitude  Longitude  First_IP_Number  Last_IP_Number
0      BW   Gaborone  -24.6464    25.9119      692781056.0     692781567.0
1      BW   Gaborone  -24.6464    25.9119      692781824.0     692783103.0
2      BW   Gaborone  -24.6464    25.9119      692909056.0     692909311.0
3      BW   Gaborone  -24.6464    25.9119      692909568.0     692910079.0
4      BW   Gaborone  -24.6464    25.9119      693051392.0     693052415.0


In [16]:
VisitorData.shape

(1247464, 6)

In [17]:
sView='Assess_Visitor_UseIt'
print('Creating :',sDatabaseName,' View:',sView)
sSQL="DROP VIEW IF EXISTS " + sView + ";"
sql.execute(sSQL,conn)

Creating : C:/VKHCG/02-Krennwallner/02-Assess/SQLite/krennwallner.db  View: Assess_Visitor_UseIt


In [18]:
sSQL="CREATE VIEW " + sView + " AS"
sSQL=sSQL+ " SELECT"
sSQL=sSQL+ " A.Country,"
sSQL=sSQL+ " A.Place_Name,"
sSQL=sSQL+ " A.Latitude,"
sSQL=sSQL+ " A.Longitude,"
sSQL=sSQL+ " (A.Last_IP_Number - A.First_IP_Number) AS UsesIt"
sSQL=sSQL+ " FROM"
sSQL=sSQL+ " Assess_Visitor as A"
sSQL=sSQL+ " WHERE"
sSQL=sSQL+ " Country is not null"
sSQL=sSQL+ " AND"
sSQL=sSQL+ " Place_Name is not null;"
sql.execute(sSQL,conn)

In [19]:
sView='Assess_Total_Visitors_Location'
print('Creating :',sDatabaseName,' View:',sView)
sSQL="DROP VIEW IF EXISTS " + sView + ";"
sql.execute(sSQL,conn)

Creating : C:/VKHCG/02-Krennwallner/02-Assess/SQLite/krennwallner.db  View: Assess_Total_Visitors_Location


In [20]:
sSQL="CREATE VIEW " + sView + " AS"
sSQL=sSQL+ " SELECT"
sSQL=sSQL+ " Country,"
sSQL=sSQL+ " Place_Name,"
sSQL=sSQL+ " SUM(UsesIt) AS TotalUsesIt"
sSQL=sSQL+ " FROM"
sSQL=sSQL+ " Assess_Visitor_UseIt"
sSQL=sSQL+ " GROUP BY"
sSQL=sSQL+ " Country,"
sSQL=sSQL+ " Place_Name"
sSQL=sSQL+ " ORDER BY"
sSQL=sSQL+ " TotalUsesIt DESC"
sSQL=sSQL+ " LIMIT 10;"
sql.execute(sSQL,conn)

In [21]:
sView='Assess_Total_Visitors_GPS'
print('Creating :',sDatabaseName,' View:',sView)
sSQL="DROP VIEW IF EXISTS " + sView + ";"
sql.execute(sSQL,conn)

Creating : C:/VKHCG/02-Krennwallner/02-Assess/SQLite/krennwallner.db  View: Assess_Total_Visitors_GPS


In [22]:
sSQL="CREATE VIEW " + sView + " AS"
sSQL=sSQL+ " SELECT"
sSQL=sSQL+ " Latitude,"
sSQL=sSQL+ " Longitude,"
sSQL=sSQL+ " SUM(UsesIt) AS TotalUsesIt"
sSQL=sSQL+ " FROM"
sSQL=sSQL+ " Assess_Visitor_UseIt"
sSQL=sSQL+ " GROUP BY"
sSQL=sSQL+ " Latitude,"
sSQL=sSQL+ " Longitude"
sSQL=sSQL+ " ORDER BY"
sSQL=sSQL+ " TotalUsesIt DESC"
sSQL=sSQL+ " LIMIT 10;"
sql.execute(sSQL,conn)

In [23]:
sTables=['Assess_Total_Visitors_Location', 'Assess_Total_Visitors_GPS']
sTables

['Assess_Total_Visitors_Location', 'Assess_Total_Visitors_GPS']

In [24]:
for sTable in sTables:
    print('################')  
    print('Loading :',sDatabaseName,' Table:',sTable)
    sSQL=" SELECT "
    sSQL=sSQL+ " *"
    sSQL=sSQL+ " FROM"
    sSQL=sSQL+ " " + sTable + ";"
    TopData=pd.read_sql_query(sSQL, conn)
    print('################')  
    print(TopData)
    print('################')  
    print('################################')
    print('Rows : ',TopData.shape[0])
    print('########### Done #####################')

################
Loading : C:/VKHCG/02-Krennwallner/02-Assess/SQLite/krennwallner.db  Table: Assess_Total_Visitors_Location
################
  Country     Place_Name  TotalUsesIt
0      CN        Beijing   53139734.0
1      US      Palo Alto   33682341.0
2      US  Fort Huachuca   33472427.0
3      JP          Tokyo   31405249.0
4      US      Cambridge   25598851.0
5      US      San Diego   17751688.0
6      CN      Guangzhou   17563744.0
7      US         Newark   17270349.0
8      US        Raleigh   17167484.0
9      US         Durham   16914033.0
################
################################
Rows :  10
########### Done #####################
################
Loading : C:/VKHCG/02-Krennwallner/02-Assess/SQLite/krennwallner.db  Table: Assess_Total_Visitors_GPS
################
   Latitude  Longitude  TotalUsesIt
0   39.9289   116.3883   53139991.0
1   37.3762  -122.1826   33551404.0
2   31.5273  -110.3607   33472427.0
3   35.6427   139.7677   31440222.0
4   23.1167   113.2500   